In [6]:
# !pip install selenium

In [7]:
# !pip install webdriver-manager

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

---
### 오피넷 유가정보

In [9]:
# Chrome Browser과 Chrome Driver Version 확인하기
chrom_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrom_options)
driver.get("https://www.opinet.co.kr/searRgSelect.do")

In [10]:
# 현재 화면 Capture하기
# 나에게 제어되고 있는 브라우저인지 확인
driver.save_screenshot("../Data/001.png")

True

---
# 서울시 유가 정보 획득을 위한 Test

### 자치구명 목록 가져오기

In [ ]:
# 자치구명 정보 가져오기 (XPath): //*[@id="SIGUNGU_NM0"]
gu_list_raw = driver.find_element(By.XPATH, '//*[@id="SIGUNGU_NM0"]')
gu_list = gu_list_raw.find_elements(By.TAG_NAME,'option')
gu_names = [option.get_attribute('value') for option in gu_list]
gu_names.remove('')
print(gu_names)


### 조회버튼 클릭

In [ ]:
# 조회버튼 click위해 XPath 가져오기: //*[@id="searRgSelect"]/span
xpath = '//*[@id="searRgSelect"]/span'

search = driver.find_element(By.XPATH, xpath).click()
# 실행하면 조회버튼 클릭 실행됨

print(gu_names)

['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']


### 자치구명 선택해서 전달

In [ ]:
# 자치구 이름을 획득하여 1번 자치구를 실행
element = driver.find_element(By.ID, 'SIGUNGU_NM0')
# 드랍다운에 key값을 전해준다. (1번: 강동구)
element.send_keys(gu_names[1])

### 엑셀 저장버튼 클릭

In [ ]:
# 엑셀 저장버튼 클릭: //*[@id="glopopd_excel"]/span
save_path = '//*[@id="glopopd_excel"]/span'
save = driver.find_element(By.XPATH, save_path).click()

# 테스트 완료

---

# 데이터 수집
- 서울시 1번 ~ 25번 자치구별 주유소 조회 후 엑셀파일로 저장까지

In [ ]:
# 화면에 그림을 그리는 시간이 필요하므로 이를 delay 시키도록 한다.
# import time
# for gu in gu_names:
#     element = driver.find_element(By.ID, 'SIGUNGU_NM0')
#     element.send_keys(gu)
#     time.sleep(2)
    
#     xpath = '//*[@id="SIGUNGU_NM0"]'
#     gu_list_raw = driver.find_element(By.XPATH, xpath)
#     time.sleep(1)

#     xpath = '//*[@id="glopopd_excel"]/span'
#     element_get_excel = driver.find_element(By.XPATH, save_path).click()
#     time.sleep(1)

---
# 가장 맛있는 족발 매장정보
- 매장명
- 매장주소
- 전화번호
- 이미지


In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as req
import time

In [13]:
chrom_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrom_options)
driver.get("https://gajok.kr/sub/store.php")

url = "https://gajok.kr/sub/store.php"
res = req.urlopen(url)
soup = BeautifulSoup(res,"html.parser")

In [ ]:
names = []
addresses = []
tels = []
# 상세정보 버튼 클릭
for i in range(1,404):
    try:
        
        driver.find_element(By.XPATH,f'//*[@id="result_store"]/div[1]/ol/li[{i}]/a').click()
        time.sleep(1)
    
        # 정보 dialog
        info_list_raw = driver.find_element(By.XPATH, '//*[@id="map"]/div[1]/div/div[6]/div[404]')
    
        # 지점명 가져오기
        name_list = info_list_raw.find_element(By.TAG_NAME,'p').get_attribute('textContent')
        print('지점명:\t',name_list)
        names.append(name_list)

        # 주소 가져오기
        address_list = info_list_raw.find_element(By.TAG_NAME,'li').get_attribute('textContent')
        print('주소:\t',address_list)
        time.sleep(1)
        addresses.append(address_list)

        # tel_list = soup.find('#map > div > div > div > div > div > div.text > ul > li')
        tel_list_raw = driver.find_element(By.XPATH,'//*[@id="map"]/div[1]/div/div[6]/div[404]/div/div[2]/ul/li[2]')
        tel_list = tel_list_raw.get_attribute("textContent")
        print('전화번호:\t',tel_list)
        tels.append(tel_list)

        driver.find_element(By.XPATH,'//*[@id="title"]/a/img').click()
        time.sleep(1)
    except:
        print("----------\n오류\n----------")
        pass
    


In [ ]:
df = pd.DataFrame(data=zip(names,addresses,tels), columns=["Branch","Address","Tel"], index=None)
df
df.to_csv("../Data/Gajok.csv",encoding="utf-8",index=False)
df

,Branch,Address,Tel
0,강남1호점,서울시 서초구 서초대로73길 42(서초동),02-595-5849
1,선정릉점,서울특별시 강남구 봉은사로57길 5(삼성동),02-543-5849
2,한남오거리점,서울시 용산구 한남대로 21길 32(한남동),02-797-1231
3,일원점,서울시 강남구 일원로 9길 55(일원동),02-3412-0070
4,뚝섬역점,서울시 성동구 상원1길 42(성수동),02-468-7535
...,...,...,...
398,거제아주동점,경남 거제시 아주동 1688-10,055-681-8682
399,거제장승포점,경남 거제시 능포동 624-2,055-681-5853
400,제주삼화점,제주특별자치도 제주시 화북일동 1023-4,064-722-5849
401,제주이도점,제주특별자치도 제주시 승천로72(아라이동),064-723-5849
